In [23]:
import pandas as pd
from pathlib import Path
from prophet import Prophet
import logging
import cmdstanpy
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
logging.getLogger('prophet').setLevel(logging.WARNING)

In [24]:
def timeseriesmodelprediction(region: str, option: str = 'Parramatta'):
    DF = pd.DataFrame(columns=['date', 'ons', 'offs', 'total'])
    folder = Path('OpalPatronage')
    ind = 0
    for file in folder.glob('*.txt'):
        try:
            df = pd.read_csv(file, sep='|')
            df = df[df['ti_region'] == 'Parramatta']
            df[['Tap_Ons', 'Tap_Offs']] = df[['Tap_Ons', 'Tap_Offs']].replace({'<50': 25, '<100': 50})
            df['tap_hour'] = df['tap_hour'].astype(int)
            df['Tap_Ons'] = df['Tap_Ons'].astype(int)
            df['Tap_Offs'] = df['Tap_Offs'].astype(int)
            date = pd.to_datetime(df['trip_origin_date'].unique()[0])
            if date >= pd.to_datetime('2022-01-01'):
                DF.loc[ind] = [date, df['Tap_Ons'].sum(), df['Tap_Offs'].sum(), df['Tap_Ons'].sum() + df['Tap_Offs'].sum()]
                ind += 1
        except Exception:
            pass
    DF.sort_values(by='date', inplace=True)
    DF.dropna(subset=['ons', 'offs', 'total'], inplace=True)
    df_total = DF[['date', option]].copy()
    df_total.rename(columns={'date': 'ds', option: 'y'}, inplace=True)
    df_total['ds'] = pd.to_datetime(df_total['ds'])

    model = Prophet()
    model.fit(df_total)

    future_dates = pd.date_range(start='2025-07-16', periods=169, freq='D')
    future_df = pd.DataFrame({'ds': future_dates})

    forecast = model.predict(future_df)
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].rename(
    columns={
        'ds': 'Date',
        'yhat': 'Predicted value',
        'yhat_lower': 'Lower bound',
        'yhat_upper': 'Upper bound'
    }
)

In [25]:
timeseriesmodelprediction('Parramatta','ons')

,Date,Predicted value,Lower bound,Upper bound
0,2025-07-16,69791.631825,60488.610152,79752.328932
1,2025-07-17,70121.323566,60493.215469,79095.415940
2,2025-07-18,63051.347136,53199.503982,72185.346426
3,2025-07-19,44170.780358,35304.095247,52914.650559
4,2025-07-20,37406.211896,28046.727725,46958.245423
...,...,...,...,...
164,2025-12-27,32642.250605,22949.800935,42393.466860
165,2025-12-28,25092.028606,15680.586837,34551.587305
166,2025-12-29,50615.099240,41125.307458,59814.902844
167,2025-12-30,57826.016942,48083.441828,67219.609881
